In [4]:
import nest_asyncio
import re 
from pydantic import BaseModel,Field
from pydantic_ai import RunContext,Agent
from dataclasses import dataclass
from typing import List
import os
import re, urllib, requests
import numpy as np
import io 
from langchain.document_loaders import PyPDFLoader
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings


In [5]:
nest_asyncio.apply()

In [6]:
def clean_text(text):
    """Clean extracted text by removing extra whitespace and empty lines"""
    text = re.sub(r'\n\s*\n', '\n\n', text.strip())
    return text

In [7]:
def clean_dataset(entire_text: Document) -> Document:
    try:
        text = entire_text.page_content
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.compile(r'(\w+)\s*-\s*(\w+)').sub(lambda match: match.group(1) + match.group(2), text)
        entire_text.page_content = text
        return entire_text    
    except Exception as e:
        print("Error occurred at method clean_dataset: ", e)

# Load documents
    1. Ingest documents

In [8]:
def extract_text(pdfpath):    
    path_to_files = pdfpath
    pdfs = [os.path.join(path_to_files, i) for i in os.listdir(path_to_files)]
    texts = []
    for file in tqdm(pdfs):
        print(file)
        loader = PyPDFLoader(file)
        pages = loader.load_and_split()
        pages = [clean_dataset(text.model_copy()) for text in tqdm(pages, desc="Cleaning Pages")]
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 200,separators=[
            "\n\n",
            "\n",
            " ",
            ".",
            ",",
            "\u200b",  
            "\uff0c",  
            "\u3001",  
            "\uff0e",  
            "\u3002",  
            "",
        ],
        )
        docs = text_splitter.split_documents(pages)
        return docs   

In [9]:
text = extract_text(pdfpath="Pdffiles")

  0%|          | 0/1 [00:00<?, ?it/s]

Pdffiles/attention.pdf


  0%|          | 0/1 [00:00<?, ?it/s]


In [10]:
text[:2]

[Document(metadata={'source': 'Pdffiles/attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗'),
 Document(metadata={'source': 'Pdffiles/attention.pdf', 'page': 0}, page_content='Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction m

In [11]:
def create_vector_db(text):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
    db = FAISS.from_documents(text,embeddings)
    db.save_local("db_faiss") 


In [12]:
create_vector_db(text)

/var/folders/5k/wmytnlns4sd2bm57pyry3xxr0000gn/T/ipykernel_13492/3195694990.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
/Users/ripeshghimire/coding/Chatbot/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
db = FAISS.load_local(
    "db_faiss",
    HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device': 'cpu'}
    ),
    allow_dangerous_deserialization=True
)

In [14]:
text = db.similarity_search('what is transformers')

In [24]:
from pydantic_ai import Agent
rag_agent = Agent(
    model= "groq:llama-3.2-3b-preview",
    system_prompt="You are a excellent chatbot that answers the queries of the user that answer questions according to the context given to you if you don't have the context don't give the answer"
)
rag_agent.run_sync(text)

BadRequestError: Error code: 400 - {'error': {'message': "'messages.1' : for 'role:user' the following must be satisfied[('messages.1.content' : one of the following must be satisfied[('messages.1.content' : value must be a string) OR ('messages.1.content.0' : one of the following must be satisfied[('messages.1.content.0' : value must be an object) OR ('messages.1.content.0' : value must be an object)])])]", 'type': 'invalid_request_error'}}